<a href="https://colab.research.google.com/github/UshanAlwis98/Shopping-Mart-Analytics/blob/main/Shopping_Mart_Analytics_Gold_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Shopping Mart Structured data upload**

In [1]:
from google.colab import files
uploaded = files.upload()

Saving customers.csv to customers.csv
Saving Orders_Data.csv to Orders_Data.csv
Saving products.csv to products.csv


# **Shopping Mart Unstructured data upload**

In [2]:
from google.colab import files
uploaded = files.upload()

Saving reviews.json to reviews.json
Saving social_media.json to social_media.json
Saving web_logs.json to web_logs.json


# **Load Silver Data**

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create Spark session
spark = SparkSession.builder \
    .appName("ShoppingMartAnalysis") \
    .getOrCreate()

# Load data properly
Orders_df = spark.read.format("csv").option("header", "true").load("/content/Orders_Data.csv")

reviews_df = spark.read.json("/content/reviews.json")
social_df = spark.read.json("/content/social_media.json")
weblogs_df = spark.read.json("/content/web_logs.json")

# Show data
Orders_df.show(5)
social_df.show(5, truncate=False)



+-------+--------------------+----------+---------+--------+-----------+-------------+
|OrderID|           OrderDate|CustomerID|ProductID|Quantity|TotalAmount|PaymentMethod|
+-------+--------------------+----------+---------+--------+-----------+-------------+
|ORD1000|2022-01-01 00:00:...|   CUST185|  PROD221|       1|     429.51|       PayPal|
|ORD1001|2022-01-01 08:45:...|   CUST197|  PROD204|       4|     463.89|   Debit Card|
|ORD1002|2022-01-01 17:31:...|   CUST172|  PROD211|       3|     495.65|  Credit Card|
|ORD1003|2022-01-02 02:17:...|   CUST132|  PROD233|       3|     336.62|   Debit Card|
|ORD1004|2022-01-02 11:03:...|   CUST145|  PROD200|       3|     379.04|       PayPal|
+-------+--------------------+----------+---------+--------+-----------+-------------+
only showing top 5 rows

+------------------------------------------------------------------------------------------------------------+---------+---------+-------------+
|content                                       

# **KPI1 : Aggregates web log data to measure engagement per user on each page and action.**

In [6]:
weblogs_df = weblogs_df.groupBy("user_id", "page", "action").count()

weblogs_df.write.mode("overwrite").parquet("/content/Files/ShoppingMart_Gold_Web_Logs/ShoppingMart_web_logs")

weblogs_df.show(10, truncate=False)


+-------+-------------------------+-----------+-----+
|user_id|page                     |action     |count|
+-------+-------------------------+-----------+-----+
|CUST102|tag                      |add_to_cart|1    |
|CUST150|tags/main                |add_to_cart|1    |
|CUST167|main                     |purchase   |1    |
|CUST171|search/tag/main          |view       |1    |
|CUST116|posts/category/main      |purchase   |1    |
|CUST175|category/tag/tag         |purchase   |1    |
|CUST129|tag/category             |click      |1    |
|CUST146|search                   |purchase   |1    |
|CUST184|tags                     |click      |1    |
|CUST199|search/explore/categories|click      |1    |
+-------+-------------------------+-----------+-----+
only showing top 10 rows



# **KPI2 : Aggregates unstructured social media data to track sentiment trends across different platforms.**


In [9]:
# Read Bronze/Silver JSON data
social_df = spark.read.json("/content/social_media.json")

# Aggregate data by platform and sentiment
social_df = social_df.groupBy("platform", "sentiment").count()

# Write aggregated data to Gold layer as Parquet (folder path, not file)
social_df.write.mode("overwrite").parquet("/content/social_media_gold")

# Show result
social_df.show(10, truncate=False)



+---------+---------+-----+
|platform |sentiment|count|
+---------+---------+-----+
|Facebook |neutral  |335  |
|Instagram|neutral  |323  |
|Twitter  |neutral  |334  |
|Twitter  |negative |360  |
|Facebook |negative |339  |
|Instagram|positive |319  |
|Facebook |positive |333  |
|Instagram|negative |343  |
|Twitter  |positive |314  |
+---------+---------+-----+



# **KPI3: Aggregates product reviews to calculate the average rating per product.**

In [29]:

# Read Bronze/Silver JSON data
web_logs_df = spark.read.json("/content/web_logs.json")

# Aggregate data by platform and sentiment
web_logs_df = social_df.groupBy("platform", "sentiment").count()

# Write aggregated data to Gold layer as Parquet (folder path, not file)
web_logs_df.write.mode("overwrite").parquet("/content/web_logs.json")

# Show result
web_logs_df.show(10, truncate=False)




+---------+---------+-----+
|platform |sentiment|count|
+---------+---------+-----+
|Facebook |neutral  |1    |
|Instagram|neutral  |1    |
|Twitter  |neutral  |1    |
|Twitter  |negative |1    |
|Facebook |negative |1    |
|Instagram|positive |1    |
|Facebook |positive |1    |
|Instagram|negative |1    |
|Twitter  |positive |1    |
+---------+---------+-----+

